<a href="https://colab.research.google.com/github/SwordForShinobi/Retail-recommender-system/blob/main/RecSys_implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
!pip install implicit # if you get lucky, you"ll be able to install it like this
!sudo -H python3 -m pip install implicit --no-cache --force-reinstall --log ./implicit.txt # if you won't be so lucky
# this precautions are for windows

In [ ]:
class ModelALS:

  def __init__(self, factors=None, iterations=None):
    self.factors = factors
    self.iterations = iterations

    if self.factors is not None and self.iterations is not None:
      self.model = ALS(self.factors, self.iterations)
    else:
      raise ValueError('Please set parameters when create class')

  def prepare_matrix_to_fit(self, df=None):
    self.df = df
    if self.df is not None: # передаем сюда ЗАКОДИРОВАННЫЕ столбцы
      row = np.array(self.df[self.df.columns[3]].values.tolist()) # users
      col = np.array(self.df[self.df.columns[4]].values.tolist()) # products
      data = np.array(self.df[self.df.columns[5]].values.tolist()) # how many purchased
      self.matrix = csr((data, (row, col)), dtype=np.float32)
      return self.matrix
    else:
      raise ValueError('Please pass dataframe to work with')
  
  def fit_model(self):
    self.model.fit(self.matrix.T)

  def predict_user(self, user, n, filter_already_liked_items=True):
    '''user = particular user id(integer)
        n = amount of items to predict(integer)'''
    code_user = self.df[self.df[self.df.columns[0]] == user][self.df.columns[3]].unique()[0]
    recommendations = [t[0] for t in self.model.recommend(code_user, self.matrix, n, filter_already_liked_items)]
    recs = self.df[self.df[self.df.columns[4]].isin(recommendations)][self.df.columns[1]].unique().tolist()
    return f'For user_id={user}, we recommend next items, ids: {recs}'

  def predict_many(self, users, n, filter_already_liked_items=True):
    '''users = users you wish to predict(list)
        n = amount of items to predict for each user(integer)'''
    self.users = users
    self.preds_list = []
    users_code = self.df[self.df[self.df.columns[0]].isin(users)][self.df.columns[3]].unique().tolist()
    decode_df = self.df[[self.df.columns[4], self.df.columns[1]]].drop_duplicates() # создадим короткий дф для раскодировки. Ниже эксперементально проверенно:
    # он очень сильно ускоряет процесс
    for i in users_code:
      recommendations = [t[0] for t in self.model.recommend(i, self.matrix, n, filter_already_liked_items, recalculate_user=True)]
      self.preds_list.append(recommendations)
    return self.preds_list

  def decode_predictions(self, preds_list = None):
    self.preds_list = preds_list
    if self.preds_list is not None:
      decode_df = self.df[[self.df.columns[4], self.df.columns[1]]].drop_duplicates() # создадим короткий дф для раскодировки. Ниже экспериментально проверенно
      decode = []

      for i in self.preds_list:
        decoded_string = decode_df[decode_df['product_id(coded)'].isin(i)]['product_id'].unique().tolist()
        decode.append(decoded_string)
      # Сразу уберем пунктуацию для сохр-я и отправки на кэггл:
      characters_to_remove = '[],'
      clear_items_list = []

      for string in decode:
        for i in characters_to_remove:
          string = str(string).replace(i, '')
        clear_items_list.append(string)
  
      answer = pd.DataFrame(np.array(self.users).reshape(len(self.users), 1), columns=['user_id'])
      answer['product_id'] = clear_items_list
      return answer
    else:
      raise ValueError('Use predict_many() method 1st then pass predictions')

  def save_model(self, path=None):
    if path is not None:
      with open(path, 'wb') as directory:
        pickle.dump(self.model, directory)
    else:
      raise ValueError('Please input desirable path/filename to save to')
    
  def load_model(self, path=None):
    if path is not None:
      with open(path, 'rb') as model_probe:
        self.model = pickle.load(model_probe)
    else:
      raise ValueError('Please input desirable path/filename to load from')

In [ ]:
model = ModelALS(30, 8)

In [ ]:
matrix = model.prepare_matrix_to_fit(to_fit)

In [ ]:
model.fit_model()

In [ ]:
model.save_model('/content/drive/MyDrive/Skillbox/coursework_Recsys/Saved_models/probe(30_8)')

In [ ]:
model.load_model('/content/drive/MyDrive/Skillbox/coursework_Recsys/Saved_models/probe(30_8)')

In [ ]:
model.predict_user(2653, 10)

In [ ]:
preds = model.predict_many(to_fit['user_id'].unique().tolist(), 10)